In [1]:
def fib(n):
    assert n >= 0
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

In [2]:
[fib(n) for n in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [3]:
fib(10000)

RecursionError: maximum recursion depth exceeded in comparison

In [4]:
def fib(n):

    def helper(k, a, b):
        if k == 0:
            return a
        else:
            a2 = b
            b2 = a + b
            return helper(k - 1, a2, b2)

    assert n >= 0
    return helper(n, 0, 1)

In [5]:
[fib(n) for n in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [7]:
fib(10000)

RecursionError: maximum recursion depth exceeded in comparison

In [24]:
import functools

In [25]:
def trampoline_tailrec(f):

    @functools.wraps(f)
    def decorator(*args, **kwargs):
        result = f(*args, **kwargs)
        while callable(result):
            result = result()
        return result

    return decorator

In [26]:
def fib(n):

    @trampoline_tailrec
    def helper(k, a, b):
        if k == 0:
            return a
        else:
            a2 = b
            b2 = a + b
            return lambda: helper(k - 1, a2, b2)

    assert n >= 0
    return helper(n, 0, 1)


In [27]:
[fib(n) for n in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [28]:
fib(10000)

RecursionError: maximum recursion depth exceeded in comparison

In [20]:
def fib(n):

    def helper(k, a, b):
        if k == 0:
            return a
        else:
            a2 = b
            b2 = a + b
            return lambda: helper(k - 1, a2, b2)

    assert n >= 0
    decorated_helper = trampoline_tailrec(helper)
    return decorated_helper(n, 0, 1)


In [21]:
[fib(n) for n in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [22]:
fib(10000)

3364476487643178326662161200510754331030214846068006390656476997468008144216666236815559551363373402558206533268083615937373479048386526826304089246305643188735454436955982749160660209988418393386465273130008883026923567361313511757929743785441375213052050434770160226475831890652789085515436615958298727968298751063120057542878345321551510387081829896979161312785626503319548714021428753269818796204693609787990035096230229102636813149319527563022783762844154036058440257211433496118002309120828704608892396232883546150577658327125254609359112820392528539343462090424524892940390170623388899108584106518317336043747073790855263176432573399371287193758774689747992630583706574283016163740896917842637862421283525811282051637029808933209990570792006436742620238978311147005407499845925036063356093388383192338678305613643535189213327973290813373264265263398976392272340788292817795358057099369104917547080893184105614632233821746563732124822638309210329770164805472624384237486241145309381220656491403

In [30]:
from collections import namedtuple
from collections.abc import Callable

In [34]:
Trampoline = namedtuple('Trampoline', ['lazy_expr'])


class TrampolineCallable(Callable):
    
    def __init__(self, f):
        self.f = f
        self.depth = 0
    
    def __call__(self, *args, **kwargs):
        assert self.depth == 0, "recursively calling a function instead of trampoline"
        self.depth += 1
        try:
            result = self.f(*args, **kwargs)
        finally:
            self.depth -= 1
        while isinstance(result, Trampoline):
            result = result.lazy_expr()
        return result
    
    def trampoline(self, *args, **kwargs):
        return Trampoline(lambda: self.f(*args, **kwargs))

def trampoline_tailrec(f):
    return functools.wraps(f)(TrampolineCallable(f));


In [35]:
def fib(n):

    @trampoline_tailrec
    def helper(k, a, b):
        if k == 0:
            return a
        else:
            a2 = b
            b2 = a + b
            return helper(k - 1, a2, b2)

    assert n >= 0
    return helper(n, 0, 1)

In [37]:
fib(10000)

AssertionError: recursively calling a function instead of trampoline

In [38]:
def fib(n):

    @trampoline_tailrec
    def helper(k, a, b):
        if k == 0:
            return a
        else:
            a2 = b
            b2 = a + b
            return helper.trampoline(k - 1, a2, b2)

    assert n >= 0
    return helper(n, 0, 1)

In [39]:
[fib(n) for n in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [40]:
fib(10000)

3364476487643178326662161200510754331030214846068006390656476997468008144216666236815559551363373402558206533268083615937373479048386526826304089246305643188735454436955982749160660209988418393386465273130008883026923567361313511757929743785441375213052050434770160226475831890652789085515436615958298727968298751063120057542878345321551510387081829896979161312785626503319548714021428753269818796204693609787990035096230229102636813149319527563022783762844154036058440257211433496118002309120828704608892396232883546150577658327125254609359112820392528539343462090424524892940390170623388899108584106518317336043747073790855263176432573399371287193758774689747992630583706574283016163740896917842637862421283525811282051637029808933209990570792006436742620238978311147005407499845925036063356093388383192338678305613643535189213327973290813373264265263398976392272340788292817795358057099369104917547080893184105614632233821746563732124822638309210329770164805472624384237486241145309381220656491403